In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "DRBwc_amYdiZ"
   },
   "source": [
    "# Download and Inspect the Collection\n",
    "\n",
    "The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.\n",
    "\n",
    "Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891\n",
    "\n",
    "GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "%pip install -r requirements.txt\n"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Imports\n",
    "import os\n",
    "import pandas as pd\n",
    "import pyterrier as pt\n",
    "import transformers\n",
    "import torch\n",
    "import nltk\n",
    "import spacy\n",
    "import shutil\n",
    "import matplotlib.pyplot as plt  # plotting library\n",
    "import numpy as np\n",
    "from collections import defaultdict\n",
    "import re"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "executionInfo": {
     "elapsed": 12366,
     "status": "ok",
     "timestamp": 1762962835550,
     "user": {
      "displayName": "Georgios Peikos",
      "userId": "04834132442165285194"
     },
     "user_tz": -60
    },
    "id": "4xBdfDsPYdLA",
    "outputId": "32103be7-1880-4ccb-ea70-6800e841dec1"
   },
   "source": [
    "import os\n",
    "os.makedirs(\"data\", exist_ok=True)\n",
    "\n",
    "!curl -L \"https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true\" -o data/test.json\n",
    "!curl -L \"https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true\" -o data/train.json\n",
    "!curl -L \"https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true\" -o data/validation.json\n",
    "\n",
    "import json\n",
    "\n",
    "files = [\"data/train.json\", \"data/validation.json\", \"data/test.json\"]\n",
    "\n",
    "for path in files:\n",
    "    print(f\"\\n===== {path} =====\")\n",
    "    try:\n",
    "        with open(path, \"r\", encoding=\"utf-8\") as f:\n",
    "            # Read a few hundred characters to see what kind of JSON it is\n",
    "            head = f.read(500)\n",
    "            print(\"Preview of first 500 characters:\\n\")\n",
    "            print(head[:500])\n",
    "        # Try to load only part of the file\n",
    "        with open(path, \"r\", encoding=\"utf-8\") as f:\n",
    "            data = json.load(f)\n",
    "        if isinstance(data, list):\n",
    "            print(f\"\\nLoaded {len(data)} items (list).\")\n",
    "            print(\"Dictionary keys:\", list(data[0].keys()))\n",
    "            print(json.dumps(data[0], indent=2)[:600])\n",
    "        elif isinstance(data, dict):\n",
    "            print(\"\\nTop-level is a dictionary. Keys:\", list(data.keys()))\n",
    "            for k, v in data.items():\n",
    "                if isinstance(v, list):\n",
    "                    print(f\"Key '{k}' contains a list of {len(v)} items.\")\n",
    "                    if v:\n",
    "                        print(\"First item keys:\", list(v[0].keys()))\n",
    "                        print(json.dumps(v[0], indent=2)[:600])\n",
    "                        break\n",
    "        else:\n",
    "            print(f\"Unexpected top-level type: {type(data)}\")\n",
    "    except Exception as e:\n",
    "        print(f\"Could not parse {path} as JSON: {e}\")"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "mylmVIP9bu8y"
   },
   "source": [
    "# Create the Document Collection\n",
    "\n",
    "To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.\n",
    "\n",
    "para_id: is the id of a paragraph of a news paper page."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "executionInfo": {
     "elapsed": 17568,
     "status": "ok",
     "timestamp": 1762962853135,
     "user": {
      "displayName": "Georgios Peikos",
      "userId": "04834132442165285194"
     },
     "user_tz": -60
    },
    "id": "nxch4FUUbxRw",
    "outputId": "d86e4179-defd-49d8-8b68-172c577ed825"
   },
   "source": [
    "import json\n",
    "import os\n",
    "\n",
    "inputs = [\"data/train.json\", \"data/validation.json\", \"data/test.json\"]\n",
    "output = \"data/document_collection.json\"\n",
    "\n",
    "def load_list_or_empty(path):\n",
    "    if not os.path.exists(path) or os.path.getsize(path) == 0:\n",
    "        print(f\"Skipping {path} because it is missing or empty\")\n",
    "        return []\n",
    "    try:\n",
    "        with open(path, \"r\", encoding=\"utf-8\") as f:\n",
    "            data = json.load(f)\n",
    "        if isinstance(data, list):\n",
    "            return data\n",
    "        print(f\"Skipping {path} because it is not a list at the top level\")\n",
    "        return []\n",
    "    except json.JSONDecodeError:\n",
    "        print(f\"Skipping {path} because it is not valid JSON\")\n",
    "        return []\n",
    "\n",
    "def project(recs):\n",
    "    out = []\n",
    "    for r in recs:\n",
    "        out.append({\n",
    "            \"para_id\": r.get(\"para_id\", \"\"),\n",
    "            \"context\": r.get(\"context\", \"\"),\n",
    "            \"raw_ocr\": r.get(\"raw_ocr\", \"\"),\n",
    "            \"publication_date\": r.get(\"publication_date\", \"\")\n",
    "        })\n",
    "    return out\n",
    "\n",
    "all_recs = []\n",
    "for p in inputs:\n",
    "    recs = load_list_or_empty(p)\n",
    "    print(f\"Loaded {len(recs)} records from {p}\")\n",
    "    all_recs.extend(project(recs))\n",
    "\n",
    "# deduplicate by para_id keeping the first one seen\n",
    "uniq = {}\n",
    "for rec in all_recs:\n",
    "    pid = rec.get(\"para_id\", \"\")\n",
    "    if pid and pid not in uniq:\n",
    "        uniq[pid] = rec\n",
    "\n",
    "result = list(uniq.values())\n",
    "\n",
    "with open(output, \"w\", encoding=\"utf-8\") as f:\n",
    "    json.dump(result, f, ensure_ascii=False, indent=2)\n",
    "\n",
    "print(f\"Wrote {len(result)} records to {output}\")\n",
    "print(json.dumps(result[:3], indent=2))"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "O-9wljtri-XX"
   },
   "source": [
    "## You should check that the collection you have matches that of the paper!"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "for path in inputs:\n",
    "    with open(path, \"r\", encoding=\"utf-8\") as f:\n",
    "        data = json.load(f)\n",
    "        df_check = pd.read_json(path)\n",
    "        print(f'Shape of {path}: {df_check.shape}')"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The dimensions match the ones of the paper at https://github.com/DataScienceUIBK/ChroniclingAmericaQA"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "snY9dkltgMts"
   },
   "source": [
    "# Create the Test Queries Data Structure\n",
    "\n",
    "We keep the first 10.000 queries due to memory errors in the free colab version.\n",
    "\n",
    "To be comparable, please keep the top 10.000 queries for evaluation."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "executionInfo": {
     "elapsed": 1151,
     "status": "ok",
     "timestamp": 1762962872929,
     "user": {
      "displayName": "Georgios Peikos",
      "userId": "04834132442165285194"
     },
     "user_tz": -60
    },
    "id": "7ZOmr1qBgRxi",
    "outputId": "1a4cbaaa-2813-4814-e0de-aee5aab98f7c"
   },
   "source": [
    "import json\n",
    "import re\n",
    "import unicodedata\n",
    "import string\n",
    "\n",
    "input_file = \"data/test.json\"\n",
    "output_file = \"data/test_queries.json\"\n",
    "\n",
    "# Load the data\n",
    "with open(input_file, \"r\", encoding=\"utf-8\") as f:\n",
    "    data = json.load(f)\n",
    "\n",
    "def clean_question(text):\n",
    "    if not isinstance(text, str):\n",
    "        return \"\"\n",
    "    text = unicodedata.normalize(\"NFKC\", text)\n",
    "    text = re.sub(rf\"[{re.escape(string.punctuation)}]\", \" \", text)  # remove punctuation\n",
    "    text = re.sub(r\"\\s+\", \" \", text)  # collapse multiple spaces\n",
    "    return text.strip()\n",
    "\n",
    "# Extract and clean\n",
    "queries = [\n",
    "    {\n",
    "        \"query_id\": item.get(\"query_id\", \"\"),\n",
    "        \"question\": clean_question(item.get(\"question\", \"\")),\n",
    "    }\n",
    "    for item in data\n",
    "]\n",
    "\n",
    "# Sort by query_id (assuming numeric)\n",
    "queries = sorted(queries, key=lambda x: int(x[\"query_id\"]) if str(x[\"query_id\"]).isdigit() else x[\"query_id\"])\n",
    "\n",
    "# Keep only the first 10,000\n",
    "queries = queries[:10000]\n",
    "\n",
    "# Save new JSON\n",
    "with open(output_file, \"w\", encoding=\"utf-8\") as f:\n",
    "    json.dump(queries, f, ensure_ascii=False, indent=2)\n",
    "\n",
    "print(f\"Saved {len(queries)} entries to {output_file}\")\n",
    "print(json.dumps(queries[:3], indent=2))"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "6NyCV6oqjFS0"
   },
   "source": [
    "# Create the Qrels for the test set"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "executionInfo": {
     "elapsed": 742,
     "status": "ok",
     "timestamp": 1762962873672,
     "user": {
      "displayName": "Georgios Peikos",
      "userId": "04834132442165285194"
     },
     "user_tz": -60
    },
    "id": "Lxms9bHpjIcn",
    "outputId": "26e9db71-b590-4f5f-94db-484d857db80c"
   },
   "source": [
    "input_file = \"data/test.json\"\n",
    "qrels_file = \"data/test_qrels.json\"\n",
    "answers_file = \"data/test_query_answers.json\"\n",
    "\n",
    "# Load the data\n",
    "with open(input_file, \"r\", encoding=\"utf-8\") as f:\n",
    "    data = json.load(f)\n",
    "\n",
    "# Build the qrels file: query_id, iteration=0, para_id, relevance=1\n",
    "qrels = [\n",
    "    {\n",
    "        \"query_id\": item.get(\"query_id\", \"\"),\n",
    "        \"iteration\": 0,\n",
    "        \"para_id\": item.get(\"para_id\", \"\"),\n",
    "        \"relevance\": 1\n",
    "    }\n",
    "    for item in data\n",
    "]\n",
    "\n",
    "# Build the query_answers file: same plus answer and org_answer\n",
    "query_answers = [\n",
    "    {\n",
    "        \"query_id\": item.get(\"query_id\", \"\"),\n",
    "        \"iteration\": 0,\n",
    "        \"para_id\": item.get(\"para_id\", \"\"),\n",
    "        \"relevance\": 1,\n",
    "        \"answer\": item.get(\"answer\", \"\"),\n",
    "        \"org_answer\": item.get(\"org_answer\", \"\")\n",
    "    }\n",
    "    for item in data\n",
    "]\n",
    "\n",
    "# Save both files\n",
    "with open(qrels_file, \"w\", encoding=\"utf-8\") as f:\n",
    "    json.dump(qrels, f, ensure_ascii=False, indent=2)\n",
    "\n",
    "with open(answers_file, \"w\", encoding=\"utf-8\") as f:\n",
    "    json.dump(query_answers, f, ensure_ascii=False, indent=2)\n",
    "\n",
    "print(f\"Saved {len(qrels)} entries to {qrels_file}\")\n",
    "print(f\"Saved {len(query_answers)} entries to {answers_file}\")\n",
    "print(\"Sample qrels entry:\", qrels[0])\n",
    "print(\"Sample query_answers entry:\", query_answers[0])"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "M7vkoP010nIF"
   },
   "source": [
    "# Retrieval"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Extract data from json files"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "input_files = ['data/document_collection.json', 'data/test.json', 'data/test_qrels.json', 'data/test_queries.json', 'data/test_query_answers.json', 'data/train.json', 'data/validation.json']\n",
    "\n",
    "dataframes = {}\n",
    "for input_file in input_files:\n",
    "    with open(input_file, \"r\", encoding=\"utf-8\") as f:\n",
    "        data = json.load(f)\n",
    "        dataframes[input_file] = pd.read_json(input_file)"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's visualize data and analyze them"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "dataframes['data/document_collection.json']"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "dataframes['data/train.json']"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**NOTE: in `data/document_collection.json` the rows are already deduplicated**"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### _Preprocessing_"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### **Linguistic Processing**"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "##### Normalization\n",
    "We lowercase everything and remove all special characters/tags"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--> 1st step normalization"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def normalize_text1(text):\n",
    "    if not isinstance(text, str):\n",
    "        return text\n",
    "    text = unicodedata.normalize('NFKC', text)\n",
    "    text = re.sub(r'<[^>]+>', ' ', text) # HTML\n",
    "    text = re.sub(r'\\s+', ' ', text).strip() # multiple white spaces\n",
    "    return text\n",
    "\n",
    "# in caso togliessimo la NER vanno tolti i commenti nella funzione qui sopra\n",
    "\n",
    "docColl = dataframes['data/document_collection.json']\n",
    "docColl_contNorm1 = docColl['context'].apply(normalize_text1)\n",
    "docColl_ocrNorm1 = docColl['raw_ocr'].apply(normalize_text1)\n",
    "docColl_Norm1 = docColl.copy()"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "docColl_Norm1['context'] = docColl_contNorm1\n",
    "docColl_Norm1['raw_ocr'] = docColl_ocrNorm1\n",
    "docColl_Norm1.head(25)"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "docColl['context'].compare(docColl_Norm1['context'])"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "print(docColl['context'].iloc[2])\n",
    "print(docColl_Norm1['context'].iloc[2])"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "##### NER\n",
    "We want to identify named-entities before lemmatizing the text, so that we do not lose any entity by \"shrinking\" words to their base forms."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import os\n",
    "import torch\n",
    "import pandas as pd\n",
    "from transformers import AutoTokenizer, pipeline\n",
    "from tqdm import tqdm\n",
    "\n",
    "# Settings per far runnare su gpu (se possibile)\n",
    "os.environ[\"PYTORCH_ENABLE_MPS_FALLBACK\"] = \"1\"\n",
    "device = \"mps\" if torch.backends.mps.is_available() else \"cpu\"\n",
    "\n",
    "MODEL_NAME = \"impresso-project/ner-stacked-bert-multilingual-light\"\n",
    "tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)\n",
    "\n",
    "ner_pipeline = pipeline(\n",
    "    model=MODEL_NAME,\n",
    "    tokenizer=tokenizer,\n",
    "    trust_remote_code=True,\n",
    "    device=device)\n",
    "\n",
    "def run_impresso_ner(text_series):\n",
    "    results = []\n",
    "    for text in tqdm(text_series): # tqdm per vedere i progressi nelle ore di run\n",
    "        text_str = str(text)\n",
    "        if not text_str.strip(): # per testi vuoti\n",
    "            results.append([])\n",
    "            continue\n",
    "\n",
    "        words = text_str.split()\n",
    "\n",
    "        try:\n",
    "            entities = ner_pipeline(text_str, tokens=words)\n",
    "            results.append(entities)\n",
    "        except Exception as e:\n",
    "            print(f\"Errore su un documento: {e}\")\n",
    "            results.append([]) # per non farlo bloccare se ha un errore\n",
    "    return results\n",
    "\n",
    "OUTPUT_FILE = \"data/ner_results_cache.parquet\"\n",
    "if os.path.exists(OUTPUT_FILE):\n",
    "    cached_data = pd.read_parquet(OUTPUT_FILE)\n",
    "\n",
    "    docColl_Norm1['ner_entities_context'] = cached_data['ner_entities_context']\n",
    "    docColl_Norm1['ner_entities_ocr'] = cached_data['ner_entities_ocr']\n",
    "\n",
    "else:\n",
    "    # context\n",
    "    docColl_Norm1['ner_entities_context'] = run_impresso_ner(docColl_Norm1['context'])\n",
    "    # OCR\n",
    "    docColl_Norm1['ner_entities_ocr'] = run_impresso_ner(docColl_Norm1['raw_ocr'])\n",
    "    # salvataggio su file esterno\n",
    "    docColl_Norm1[['ner_entities_context', 'ner_entities_ocr']].to_parquet(OUTPUT_FILE)"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "docColl_Norm1"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "docColl_ner = docColl_Norm1.copy()\n",
    "docColl_ner[['context', 'raw_ocr', 'ner_entities_context', 'ner_entities_ocr']]"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--> 2nd step normalization"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def normalize_text2(text):\n",
    "    if not isinstance(text, str):\n",
    "        return text\n",
    "    text = text.lower() # lowercase\n",
    "    text = re.sub(r'[^a-z0-9\\s]', '', text) # punctuations\n",
    "    text = re.sub(r'[^\\w\\s]', ' ', text) # any other punctuation mark\n",
    "    text = re.sub(r'\\s+', ' ', text).strip() # white spaces again\n",
    "    return text\n",
    "\n",
    "docColl_ner['context'] = docColl_ner['context'].apply(normalize_text2)\n",
    "docColl_ner['raw_ocr'] = docColl_ner['raw_ocr'].apply(normalize_text2)\n",
    "docColl_Norm2 = docColl_ner.copy()"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "metadata": {},
   "cell_type": "code",
   "source": "docColl_Norm2['context'].iloc[2]",
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "##### Lemmatization\n",
    "Placed here to standardize semantically the sentences in the documents"
   ]
  },
  {
   "metadata": {},
   "cell_type": "code",
   "source": [
    "import pandas as pd\n",
    "import spacy\n",
    "import os\n",
    "import gc\n",
    "from tqdm import tqdm\n",
    "\n",
    "OUTPUT_FILE_LEMM = \"data/lemmatization_results_cache.parquet\"\n",
    "columns_to_process = ['context', 'raw_ocr']\n",
    "\n",
    "# 1. Caricamento ottimizzato: disabilitiamo tutto ciò che non serve alla lemmatizzazione\n",
    "# Teniamo il 'tagger' e 'attribute_ruler' perché necessari per lemmi accurati\n",
    "try:\n",
    "    nlp = spacy.load(\"en_core_web_sm\", disable=['parser', 'ner'])\n",
    "except OSError:\n",
    "    from spacy.cli import download\n",
    "    download(\"en_core_web_sm\")\n",
    "    nlp = spacy.load(\"en_core_web_sm\", disable=['parser', 'ner'])\n",
    "\n",
    "def run_lemmatization(df, columns):\n",
    "    temp_df = pd.DataFrame(index=df.index)\n",
    "\n",
    "    for col in columns:\n",
    "        if col in df.columns:\n",
    "            print(f\"\\n--- Elaborazione colonna: '{col}' ---\")\n",
    "\n",
    "            # 2. Utilizziamo un generatore per non duplicare i dati in memoria\n",
    "            def text_generator():\n",
    "                for text in df[col]:\n",
    "                    yield str(text) if pd.notnull(text) else \"\"\n",
    "\n",
    "            processed_texts = []\n",
    "\n",
    "            # 3. Riduciamo n_process o rimuoviamolo se la RAM è poca.\n",
    "            # Aumentiamo il batch_size per compensare la velocità.\n",
    "            for doc in tqdm(nlp.pipe(text_generator(), batch_size=1000),\n",
    "                            total=len(df),\n",
    "                            desc=f\"Lemmatizing {col}\"):\n",
    "                # Usiamo una list comprehension rapida\n",
    "                lemmas = \" \".join([token.lemma_ for token in doc if not token.is_space])\n",
    "                processed_texts.append(lemmas)\n",
    "\n",
    "            temp_df[f\"{col}_lemma\"] = processed_texts\n",
    "\n",
    "            # 4. Pulizia manuale della memoria tra una colonna e l'altra\n",
    "            del processed_texts\n",
    "            gc.collect()\n",
    "        else:\n",
    "            print(f\" Avviso: Colonna '{col}' non trovata.\")\n",
    "\n",
    "    return temp_df\n",
    "\n",
    "if os.path.exists(OUTPUT_FILE_LEMM):\n",
    "    cached_lemm = pd.read_parquet(OUTPUT_FILE_LEMM)\n",
    "    docColl_Norm2['context_lemma'] = cached_lemm['context_lemma']\n",
    "    docColl_Norm2['raw_ocr_lemma'] = cached_lemm['raw_ocr_lemma']\n",
    "\n",
    "else:\n",
    "    # context\n",
    "    lemmatized_df = run_lemmatization(docColl_Norm2[['context', 'raw_ocr']], columns_to_process)\n",
    "    docColl_Norm2['context_lemma'] = lemmatized_df['context_lemma']\n",
    "    docColl_Norm2['raw_ocr_lemma'] = lemmatized_df['raw_ocr_lemma']\n",
    "    # salvataggio su file esterno\n",
    "    docColl_Norm2[['context_lemma', 'raw_ocr_lemma']].to_parquet(OUTPUT_FILE_LEMM)\n",
    "\n",
    "docColl_Lemm = docColl_Norm2"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "metadata": {},
   "cell_type": "code",
   "source": "docColl_Lemm['context_lemma'].iloc[0]",
   "outputs": [],
   "execution_count": null
  },
  {
   "metadata": {},
   "cell_type": "code",
   "source": "docColl_Lemm['context'].iloc[0]",
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "##### N-gram based tokenization\n",
    "Important to place it after normalization, in this tokenization can be integrated a NER-aware part so that \"the tokenization is also entity-guided\""
   ]
  },
  {
   "metadata": {},
   "cell_type": "code",
   "source": [
    "def ner_aware_string_processor(row, text_col, ner_col):\n",
    "\n",
    "    text = row.get(text_col, \"\")\n",
    "    entities = row.get(ner_col, [])\n",
    "\n",
    "    if not isinstance(text, str) or not text.strip():\n",
    "        return \"\"\n",
    "\n",
    "    if isinstance(entities, list) and len(entities) > 0:\n",
    "        entity_texts = sorted(\n",
    "            [ent['surface'].lower().strip() for ent in entities if ent.get('confidence_ner', 0) >= 0.5],\n",
    "            key=len,\n",
    "            reverse=True)\n",
    "\n",
    "        for ent_text in entity_texts:\n",
    "            if \" \" in ent_text:\n",
    "                glued_ent = ent_text.replace(\" \", \"_\")\n",
    "                text = text.replace(ent_text, glued_ent)\n",
    "\n",
    "    tokens = text.split()\n",
    "    return \" \".join(tokens)\n",
    "\n",
    "docColl_tok = docColl_Lemm.copy()\n",
    "\n",
    "columns_map = [\n",
    "    ('context_lemma', 'ner_entities_context', 'context_bigrams'),\n",
    "    ('raw_ocr_lemma', 'ner_entities_ocr',     'raw_ocr_bigrams')\n",
    "]\n",
    "\n",
    "for text_col, ner_col, new_col in columns_map:\n",
    "    if text_col in docColl_tok.columns and ner_col in docColl_tok.columns:\n",
    "\n",
    "        docColl_tok[new_col] = docColl_tok.progress_apply(\n",
    "            lambda row: ner_aware_string_processor(row, text_col, ner_col),\n",
    "            axis=1 )"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "metadata": {},
   "cell_type": "code",
   "source": "docColl_tok['context_bigrams'].iloc[0]",
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "da qui dovrebbe uscire il dataframe chiamato docColl_tok"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### _Multi-field Indexing_"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--> Indexing con PyTerrier usando un generator"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# qui assumiamo che le celle create dal NER siano oggetti di tipo dizionario\n",
    "def createGenerator(df, context=True):\n",
    "    # context\n",
    "    if context:\n",
    "        for _, row in df.iterrows():\n",
    "            # togliamo lOffset and rOffset\n",
    "            clean_ents = []\n",
    "            for ent in row['ner_entities_context']:\n",
    "                cleaned = {k: v for k, v in ent.items() if k not in ['lOffset', 'rOffset']}\n",
    "                clean_ents.append(cleaned)\n",
    "\n",
    "            search_terms = []\n",
    "            for e in clean_ents:\n",
    "                #search_terms.append(e.get('name', ''))\n",
    "                #search_terms.append(e.get('title', ''))\n",
    "                # da capire se vogliamo che siano searchable, dato che surface contiene già il testo a cui è associata la entity\n",
    "                search_terms.append(e.get('surface', ''))\n",
    "\n",
    "            ent_text = \" \".join(filter(None, search_terms)) # questa riga ha senso solo se prendiamo anche 'name' e 'title'\n",
    "                                                                                           # se no ent_text va assegnato a e.get('surface', ' ')\n",
    "\n",
    "            meta_json = json.dumps(clean_ents) # facciamo diventare tutti i metadati una stringa in forma json (non un oggetto dizionario, proprio una stringa)\n",
    "\n",
    "            yield { # serve per lo stream dei dati quando viene chiamata createGenerator dentro indexer.index(•)\n",
    "                \"docno\": str(row['para_id']),\n",
    "                \"text\": row['context_bigrams'],\n",
    "                \"entities\": ent_text, # entità searchable\n",
    "                \"entity_json\": meta_json}\n",
    "    # OCR\n",
    "    if not context:\n",
    "        for _, row in df.iterrows():\n",
    "            # togliamo lOffset and rOffset\n",
    "            clean_ents = []\n",
    "            for ent in row['ner_entities_ocr']:\n",
    "                cleaned = {k: v for k, v in ent.items() if k not in ['lOffset', 'rOffset']}\n",
    "                clean_ents.append(cleaned)\n",
    "\n",
    "            search_terms = []\n",
    "            for e in clean_ents:\n",
    "                #search_terms.append(e.get('name', ''))\n",
    "                #search_terms.append(e.get('title', ''))\n",
    "                # da capire se vogliamo che siano searchable, dato che surface contiene già il testo a cui è associata la entity\n",
    "                search_terms.append(e.get('surface', ''))\n",
    "\n",
    "            ent_text = \" \".join(filter(None, search_terms)) # questa riga ha senso solo se prendiamo anche 'name' e 'title'\n",
    "                                                                                           # se no ent_text va assegnato a e.get('surface', ' ')\n",
    "\n",
    "            meta_json = json.dumps(clean_ents) # facciamo diventare tutti i metadati una stringa in forma json (non un oggetto dizionario, proprio una stringa)\n",
    "\n",
    "            yield { # serve per lo stream dei dati quando viene chiamata createGenerator dentro indexer.index(•)\n",
    "                \"docno\": str(row['para_id']),\n",
    "                \"text\": row['raw_ocr_bigrams'],\n",
    "                \"entities\": ent_text, # entità searchable\n",
    "                \"entity_json\": meta_json}\n",
    "\n",
    "contextIndex_path = 'data/docColl_context-index'\n",
    "ocrIndex_path = 'data/docColl_ocr-index'"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "if os.path.exists(contextIndex_path):\n",
    "    shutil.rmtree(contextIndex_path)\n",
    "\n",
    "indexerCont = pt.IterDictIndexer(\n",
    "    'entity_index',\n",
    "    fields=['text', 'entities'],\n",
    "    meta={'docno', 'entity_json'})\n",
    "\n",
    "indexrefCont = indexerCont.index(createGenerator(docColl_tok, context=True))"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "if os.path.exists(ocrIndex_path):\n",
    "    shutil.rmtree(ocrIndex_path)\n",
    "\n",
    "indexerOCR = pt.IterDictIndexer(\n",
    "    'entity_index',\n",
    "    fields=['text', 'entities'],\n",
    "    meta={'docno', 'entity_json'})\n",
    "\n",
    "indexrefOCR = indexerOCR.index(createGenerator(docColl_tok, context=False))"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Statistics about the indexed documents"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "indexCont = pt.IndexFactory.of(indexrefCont)\n",
    "stats = indexCont.getCollectionStatistics()\n",
    "print('Index folder:', contextIndex_path)\n",
    "print('Number of documents:', stats.getNumberOfDocuments())\n",
    "print('Number of postings:', stats.getNumberOfPostings())\n",
    "print('Number of tokens:', stats.getNumberOfTokens())\n",
    "print('Number of unique terms:', stats.getNumberOfUniqueTerms())\n",
    "print('Average document length:', stats.getAverageDocumentLength())"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "indexOCR = pt.IndexFactory.of(indexrefOCR)\n",
    "stats = indexOCR.getCollectionStatistics()\n",
    "print('Index folder:', contextIndex_path)\n",
    "print('Number of documents:', stats.getNumberOfDocuments())\n",
    "print('Number of postings:', stats.getNumberOfPostings())\n",
    "print('Number of tokens:', stats.getNumberOfTokens())\n",
    "print('Number of unique terms:', stats.getNumberOfUniqueTerms())\n",
    "print('Average document length:', stats.getAverageDocumentLength())"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Query analysis"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "display(queries.head(10))"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--> da scrivere commento riguardo l'analisi delle queries"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Qrels analysis"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "display(qrels.sample(10))"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# stats for the qrels\n",
    "# Count how many relevance assessments each query has\n",
    "counts = qrels.groupby(\"query_id\")[\"para_id\"].count()  # group by query id and count documents\n",
    "print('Overall Statistics')\n",
    "print(counts.describe())  # show a summary of the count distribution\n",
    "\n",
    "# Plot how many relevance assessments each query received\n",
    "plt.figure()  # create a new figure\n",
    "counts.plot(kind='hist')  # histogram showing distribution of judgment counts\n",
    "plt.xlabel('Number of relevance assessments per query')  # label for x-axis\n",
    "plt.ylabel('Number of queries')  # label for y-axis\n",
    "plt.title('Relevance assessment distribution')  # title of the plot\n",
    "plt.show()  # display the plot\n",
    "\n",
    "# Show the queries with the highest number of relevance assessments\n",
    "counts.sort_values(ascending=False).head()  # top queries by number of judgments\n",
    "\n",
    "# Count how many times each relevance label occurs overall\n",
    "qrels['relevance'].value_counts()  # distribution of relevance scores (e.g., 0, 1, 2, etc.)\n",
    "\n",
    "# Plot the label distribution as a histogram\n",
    "plt.figure()  # create a new figure\n",
    "qrels['relevance'].plot(kind='hist')  # histogram of relevance labels\n",
    "plt.xlabel('Relevance score')  # label for x-axis\n",
    "plt.ylabel('Frequency')  # label for y-axis\n",
    "plt.title('Relevance score distribution')  # title of the plot\n",
    "plt.show()  # display the plot"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--> commento riguardo l'analisi delle qrels"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Phase I - Topical relevance-based retrieval"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### **BM25 Retrieval from raw OCR (baseline 1)**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "bm25ocr = pt.terrier.Retriever(indexrefOCR, wmodel='BM25', ) # dovremmo usare un BM25F? per dividere i fields di ricerca (secondo me si)\n",
    "res_bm25ocr = bm25ocr.transform()"
   ],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### **BM25 Retrieval from corrected OCR (baseline 2)**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [],
   "outputs": [],
   "execution_count": null
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### **BM25 Retrieval from both raw and corrected OCR using RRF formula (baseline 3)**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [],
   "outputs": [],
   "execution_count": null
  }
 ],
 "metadata": {
  "colab": {
   "authorship_tag": "ABX9TyNUuc82OtGicqd8vHTH8YSN",
   "provenance": [],
   "toc_visible": true
  },
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}


NameError: name 'null' is not defined